In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# CAN CHANGE THE FOLLOWING
dataset = 1
regenerate_sawo_sample = False # > 20 minutes if True, < 2 minutes if False
# CHANGE ANYTHING ELSE AT YOUR OWN RISK


import sys
sys.path.insert(0,"/content/drive/MyDrive/Colab Notebooks/TestSAWO/")

from Rule import Rule
from RuleSet import RuleSet
from Node import Node
from Architecture import Architecture
from SawoNN import SawoNN
from Member import Member

import copy
import math
import time
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers as layers
from sklearn.model_selection import train_test_split
import statistics as stats
import matplotlib.pyplot as plt

folder_path = "/content/drive/MyDrive/Colab Notebooks/TestSAWO/"
dataset_path = folder_path + "datasets/"

rs = RuleSet(folder_path + "grammar.txt")
print(rs)


X = np.load(dataset_path + str(dataset) + '_input.npy')
print("X.shape:", X.shape)
y = np.load(dataset_path + str(dataset) + '_target.npy')
print("y.shape:", y.shape)

sample_size = 30
num_epochs = 20
num_batches = 5

class_T_reg_F = True
if dataset >= 9:
    class_T_reg_F = False

seeds = [109031, 32228, 97483, 110457, 74949, 29240, 23745, 45647, 105953, 74080, 62741, 60320]
seed = seeds[dataset-1]


RULE SET
--------
START := { "a": SIZE , "b": SIZE , "c": SIZE } $ self.gu ( 'io' , CONSTANT , 'mse' , ARCHITECTURE_IO ) ~ self.gu ( 'ia' , CONSTANT , 'mse' , ARCHITECTURE_IA ) ~ self.gu ( 'ib' , CONSTANT , 'mse' , ARCHITECTURE_IB ) ~ self.gu ( 'ic' , CONSTANT , 'mse' , ARCHITECTURE_IC ) ~ self.gu ( 'ab' , CONSTANT , 'mse' , ARCHITECTURE_AB ) ~ self.gu ( 'ba' , CONSTANT , 'mse' , ARCHITECTURE_BA ) ~ self.gu ( 'ac' , CONSTANT , 'mse' , ARCHITECTURE_AC ) ~ self.gu ( 'ca' , CONSTANT , 'mse' , ARCHITECTURE_CA ) ~ self.gu ( 'bc' , CONSTANT , 'mse' , ARCHITECTURE_BC ) ~ self.gu ( 'cb' , CONSTANT , 'mse' , ARCHITECTURE_CB ) ~ self.gu ( 'ao' , CONSTANT , 'mse' , ARCHITECTURE_AO ) ~ self.gu ( 'bo' , CONSTANT , 'mse' , ARCHITECTURE_BO ) ~ self.gu ( 'co' , CONSTANT , 'mse' , ARCHITECTURE_CO ) $ ARCHITECTURE
SIZE := 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20
ARCHITECTURE := ['i', 'o'] | ['i', 'a', 'o'] | ['i', 'ia', 'o'] | ['i', 'b', 'o'] | ['i', 'ib', 'o'

In [18]:
#CREATE SAWO SAMPLE

if regenerate_sawo_sample:
    sample_path = folder_path + "samples/"

    chromosomes = [None]*13
    chromosomes[1] = ['0000000', '0101101', '0001101', '0001101', '0000001', '0011010', '0000001', '0110110', '0000001', '1010111', '0001001', '0101000', '0000001', '0010001', '0000001', '0000001', '0000011', '0001000', '0001001', '0001110', '0000001', '1110011', '0000001', '1101011', '0000001', '1110100', '0000001', '0111101', '0000001', '0100101', '1010000']
    chromosomes[2] = ['0000000', '0001101', '0001001', '0001101', '0000001', '1111100', '0010001', '1110001', '0000001', '0001111', '1000001', '0100110', '0000000', '1110011', '0000001', '0001101', '0000001', '1101010', '0000101', '0001100', '0000001', '1100011', '0000001', '1010011', '0000001', '0011001', '0000001', '1110010', '0000000', '1010000', '1001110']
    chromosomes[3] = ['1001000', '0011101', '0001101', '0001000', '1000001', '0000011', '0000001', '0000101', '0000001', '1100110', '0000001', '0000001', '0000001', '1001110', '0000001', '0110100', '0000001', '1010011', '0000101', '1010001', '0000000', '1000110', '0000001', '0001101', '0000001', '0111000', '0000001', '0110011', '0000000', '0000100', '0000110']
    chromosomes[4] = ['0000001', '0001001', '0001101', '0001101', '0010001', '0101000', '0000001', '1000011', '1000001', '0110110', '0000001', '1110101', '0000001', '0111101', '0000001', '0110100', '0000001', '1100011', '0100001', '1010011', '0000101', '1110010', '0100000', '0010100', '0000001', '0100001', '0000011', '1110010', '0000001', '1101001', '1011101']
    chromosomes[5] = ['0000000', '0001101', '0001101', '0001101', '0000001', '0111000', '0000001', '0010010', '0000001', '1111011', '0000011', '0101000', '0000011', '0010001', '0000001', '1100001', '0000001', '0010111', '0000001', '0001000', '0000001', '1101111', '0000001', '0111000', '0000001', '1101111', '0010001', '0011101', '0000001', '1111000', '1010000']
    chromosomes[6] = ['0000000', '0001111', '0001111', '0001101', '0000001', '0110110', '0010101', '0001010', '1000001', '0100010', '0000001', '0101011', '0000001', '0010101', '0001001', '1011101', '0000001', '0100000', '0000000', '1111110', '0000001', '0100111', '0000001', '1100001', '0000001', '1011111', '0000001', '1100000', '0010001', '0000010', '0010001']
    chromosomes[7] = ['0010000', '0001101', '0001101', '0001101', '0000001', '1110101', '0001001', '0110110', '0000011', '0111101', '0000101', '1110100', '0000001', '1001010', '0000000', '0010011', '0000001', '1101010', '0000001', '1110101', '0000101', '0111011', '0000001', '0010011', '1000001', '0100000', '0000001', '1101001', '0000001', '1111100', '1011100']
    chromosomes[8] = ['0000000', '0001101', '0001101', '0001101', '0000001', '0100000', '1000001', '0011100', '0000001', '1001101', '0000101', '1101101', '0000001', '0101011', '0000001', '0000001', '0001001', '0000111', '0000001', '0100000', '0000001', '1111101', '0000001', '1111000', '0000001', '0111010', '0000001', '0001110', '0001001', '0011000', '0101111']
    chromosomes[9] = ['0000000', '0011101', '0001101', '0001101', '0000101', '1101010', '0000001', '0101001', '0000011', '0011110', '0000001', '0011101', '0000001', '0000111', '0000001', '1111110', '0000001', '1101110', '0010001', '1011001', '0000001', '0111010', '0000001', '0111100', '0000001', '1011011', '0010001', '0011011', '0000001', '1100011', '0111100']
    chromosomes[10] = ['0000000', '0001101', '0001101', '0001101', '0000101', '0000001', '0000101', '0000001', '0000001', '0101010', '0000001', '1110111', '0000001', '1101000', '0000001', '1011001', '0000001', '0000011', '0000001', '1000101', '0000001', '1101010', '0000001', '0001100', '0000001', '0000011', '0000001', '0110010', '0000001', '0000011', '0000010']
    chromosomes[11] = ['0000000', '0001101', '0001101', '0001101', '0001001', '0001111', '0000001', '0110110', '0000001', '0001101', '0000001', '0001011', '0000001', '1101110', '0000001', '1101111', '0000001', '0010111', '0000001', '1011000', '0001001', '0000011', '0000001', '0001101', '0000001', '1000100', '0000001', '1001100', '0001001', '0101011', '0011110']
    chromosomes[12] = ['0000000', '0001101', '0001101', '0001101', '0000001', '1110101', '0000001', '0111101', '0000011', '0011000', '0010001', '0010110', '0000001', '0001100', '0000001', '1100111', '0000001', '0001000', '0000001', '1111001', '0000001', '1111100', '0100001', '1110111', '0000001', '0100101', '0000001', '0001111', '0000001', '0011001', '0110000']
    
    chromosome = chromosomes[dataset]

    chromosome_int_version = [int(x, 2) for x in chromosome]
    hash = sum(chromosome_int_version)
    sample_filename = str(dataset) + "_" + str(hash) + ".json"


    print("dataset:", dataset)
    print("class_T_reg_F:", class_T_reg_F)
    print("seed:", seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    print("sample_size:", sample_size)
    print("num_epochs:", num_epochs)
    print("num_batches:", num_batches)
    print("chromosome:", chromosome)
    print("sample_filename:", sample_filename)

    equation = rs.constructEquation(chromosome)
    print("equation:", equation)
    print()

    meta_obj = {}
    meta_obj["dataset"] = dataset
    meta_obj["class_T_reg_F"] = class_T_reg_F
    meta_obj["seed"] = seed
    meta_obj["sample_size"] = sample_size
    meta_obj["num_epochs"] = num_epochs
    meta_obj["num_batches"] = num_batches
    meta_obj["chromosome"] = chromosome
    meta_obj["sample_filename"] = sample_filename
    meta_obj["equation"] = equation

    start_time = time.time()
    
    sample = []
    for i in range(sample_size):
        print(i)
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=(seed+i))

        best_sawo = SawoNN(equation, X_train, y_train, X_test, y_test)
        essential_eq = best_sawo.printEssentialInfo()
        print(essential_eq) 
        meta_obj["essential_eq"] = essential_eq
        p, e, w = best_sawo.train(num_epochs, num_batches, class_T_reg_F)
        if class_T_reg_F:
            print(max(e), ">", e)
            sample.append(max(e))
        else:
            print(min(e), "<", e)
            sample.append(min(e))
        print()
    
    end_time = time.time()
    
    print("sample:", sample)
    print("avg:", np.mean(sample))
    print("std:", np.std(sample))
    print("min:", min(sample))

    time_taken = (end_time - start_time)
    meta_obj["time_taken"] = time_taken

    sample = [float(x) for x in sample]
    meta_obj["sample"] = sample

    print(meta_obj)

    sample_file = open(sample_path + sample_filename, 'w')
    json.dump(meta_obj, sample_file)
    sample_file.close()

else:

    samples = [None]*13
    samples[1] = [0.9790209531784058, 0.9930070042610168, 0.9790209531784058, 0.9790209531784058, 0.9860140085220337, 0.9790209531784058, 0.9790209531784058, 0.9790209531784058, 0.9930070042610168, 0.9440559148788452, 0.9510489702224731, 0.9650349617004395, 0.9860140085220337, 0.9720279574394226, 0.9860140085220337, 0.9580419659614563, 0.9790209531784058, 0.9230769276618958, 0.9720279574394226, 0.9790209531784058, 0.9720279574394226, 0.9650349617004395, 0.9650349617004395, 0.9790209531784058, 0.9790209531784058, 0.9650349617004395, 0.9720279574394226, 0.9650349617004395, 0.9790209531784058, 0.9580419659614563]
    samples[2] = [0.9857212901115417, 0.9935992360115051, 0.9798129200935364, 0.9842442274093628, 0.9857212901115417, 0.9871984124183655, 0.9867060780525208, 0.9768586754798889, 0.9807976484298706, 0.9901526570320129, 0.9871984124183655, 0.9812899827957153, 0.9857212901115417, 0.9881831407546997, 0.9798129200935364, 0.9871984124183655, 0.9862136840820312, 0.9783357977867126, 0.9768586754798889, 0.9881831407546997, 0.9906449913978577, 0.9901526570320129, 0.9847365617752075, 0.9817823767662048, 0.9867060780525208, 0.9807976484298706, 0.9817823767662048, 0.9827671051025391, 0.9940915703773499, 0.9842442274093628]
    samples[3] = [0.8289473652839661, 0.8684210777282715, 0.7894737124443054, 0.8684210777282715, 0.9210526347160339, 0.8552631735801697, 0.8289473652839661, 0.8289473652839661, 0.9078947305679321, 0.8815789222717285, 0.8815789222717285, 0.8684210777282715, 0.8289473652839661, 0.8684210777282715, 0.9210526347160339, 0.8421052694320679, 0.8684210777282715, 0.8815789222717285, 0.8026315569877625, 0.8026315569877625, 0.8421052694320679, 0.8026315569877625, 0.8421052694320679, 0.8421052694320679, 0.8552631735801697, 0.8157894611358643, 0.8552631735801697, 0.8552631735801697, 0.8421052694320679, 0.8421052694320679]
    samples[4] = [0.9596697092056274, 0.9539536237716675, 0.956017792224884, 0.9503017067909241, 0.9518895149230957, 0.9599872827529907, 0.9615750908851624, 0.960304856300354, 0.960304856300354, 0.9595109820365906, 0.9522070288658142, 0.953159749507904, 0.9515719413757324, 0.9555414319038391, 0.9537948369979858, 0.9593521952629089, 0.9536360502243042, 0.9566528797149658, 0.957923173904419, 0.9587170481681824, 0.9526833891868591, 0.9601460695266724, 0.9550650715827942, 0.9571292400360107, 0.9583994746208191, 0.9561765789985657, 0.9577643871307373, 0.9607812166213989, 0.956017792224884, 0.9428389668464661]
    samples[5] = [0.9473684430122375, 0.8684210777282715, 0.9736841917037964, 0.8157894611358643, 0.8421052694320679, 0.8947368264198303, 0.8947368264198303, 0.8157894611358643, 0.9210526347160339, 0.8421052694320679, 0.8947368264198303, 0.9210526347160339, 0.8684210777282715, 0.9210526347160339, 0.8421052694320679, 0.8947368264198303, 0.8421052694320679, 0.8947368264198303, 0.8684210777282715, 0.8421052694320679, 0.9473684430122375, 0.8947368264198303, 0.8684210777282715, 0.8157894611358643, 0.9210526347160339, 0.9210526347160339, 0.8947368264198303, 0.8684210777282715, 0.9210526347160339, 0.8947368264198303]
    samples[6] = [0.5849999785423279, 0.5899999737739563, 0.6150000095367432, 0.574999988079071, 0.5799999833106995, 0.6025000214576721, 0.5950000286102295, 0.5699999928474426, 0.5799999833106995, 0.5849999785423279, 0.6025000214576721, 0.6299999952316284, 0.5550000071525574, 0.5849999785423279, 0.6150000095367432, 0.5774999856948853, 0.5249999761581421, 0.5824999809265137, 0.5625, 0.5674999952316284, 0.5724999904632568, 0.5699999928474426, 0.5525000095367432, 0.5450000166893005, 0.5475000143051147, 0.5774999856948853, 0.5724999904632568, 0.6000000238418579, 0.5525000095367432, 0.5824999809265137]
    samples[7] = [0.5185185074806213, 0.5925925970077515, 0.6481481194496155, 0.6851851940155029, 0.5185185074806213, 0.5740740895271301, 0.5185185074806213, 0.7222222089767456, 0.5185185074806213, 0.5925925970077515, 0.5185185074806213, 0.5555555820465088, 0.5740740895271301, 0.6296296119689941, 0.5370370149612427, 0.6481481194496155, 0.48148149251937866, 0.6851851940155029, 0.5185185074806213, 0.5925925970077515, 0.5740740895271301, 0.6296296119689941, 0.6111111044883728, 0.6481481194496155, 0.5370370149612427, 0.5925925970077515, 0.5925925970077515, 0.5185185074806213, 0.5740740895271301, 0.5925925970077515]
    samples[8] = [0.9811320900917053, 1.0, 0.9622641801834106, 0.9245283007621765, 0.9811320900917053, 0.9811320900917053, 0.9433962106704712, 0.9433962106704712, 1.0, 0.9622641801834106, 0.9433962106704712, 0.9811320900917053, 0.9245283007621765, 0.9433962106704712, 0.9056603908538818, 0.9811320900917053, 0.8867924809455872, 0.9433962106704712, 0.9811320900917053, 0.9811320900917053, 0.9245283007621765, 0.9622641801834106, 0.9811320900917053, 0.9245283007621765, 1.0, 0.9811320900917053, 0.9622641801834106, 0.9433962106704712, 0.9433962106704712, 0.9433962106704712]
    samples[9] = [0.0038444960955530405, 0.004021282307803631, 0.00466515589505434, 0.0042761280201375484, 0.00535852275788784, 0.004053998738527298, 0.003156882245093584, 0.003349059261381626, 0.005660925060510635, 0.004377670586109161, 0.004045296460390091, 0.003996364772319794, 0.0036011403426527977, 0.005279158242046833, 0.005000901874154806, 0.00457150349393487, 0.0037600023206323385, 0.004560026340186596, 0.00254883267916739, 0.0033834478817880154, 0.0032822273205965757, 0.0042167846113443375, 0.0054353149607777596, 0.0046803574077785015, 0.004265779163688421, 0.004946419969201088, 0.004388281609863043, 0.005119765177369118, 0.003816446755081415, 0.0042976196855306625]
    samples[10] = [0.012395686469972134, 0.01830693706870079, 0.020129794254899025, 0.015355817973613739, 0.014401030726730824, 0.01795993372797966, 0.008753811940550804, 0.015638824552297592, 0.011573820374906063, 0.00993205327540636, 0.012894046492874622, 0.014218657277524471, 0.01327311061322689, 0.012275966815650463, 0.013066870160400867, 0.009632406756281853, 0.012765273451805115, 0.011667943559587002, 0.008482107892632484, 0.02472653053700924, 0.017473148182034492, 0.007742130663245916, 0.01062890887260437, 0.01846466213464737, 0.018208634108304977, 0.011252416297793388, 0.014035405591130257, 0.0084536112844944, 0.0099562993273139, 0.01940983347594738]
    samples[11] = [0.007954437285661697, 0.007609899155795574, 0.008145579136908054, 0.008067002519965172, 0.007648859638720751, 0.007545954547822475, 0.0077608563005924225, 0.008608642034232616, 0.007512789219617844, 0.007306179031729698, 0.007891870103776455, 0.007011138368397951, 0.007741579320281744, 0.008076297119259834, 0.007449021562933922, 0.007832806557416916, 0.008440663106739521, 0.00880969874560833, 0.008151650428771973, 0.007624266669154167, 0.007255620788782835, 0.0087488554418087, 0.008500288240611553, 0.00827966257929802, 0.007494653575122356, 0.009066271595656872, 0.00850632507354021, 0.00817562360316515, 0.007817498408257961, 0.008321051485836506]
    samples[12] = [0.014030450023710728, 0.027764784172177315, 0.011500739492475986, 0.014051890932023525, 0.021162137389183044, 0.02263091504573822, 0.020236315205693245, 0.018373729661107063, 0.016948269680142403, 0.014089310541749, 0.019711073487997055, 0.016373910009860992, 0.0204019732773304, 0.021586328744888306, 0.017643433064222336, 0.01911860704421997, 0.012309441342949867, 0.008489198982715607, 0.015538815408945084, 0.02699381299316883, 0.01638675667345524, 0.018052229657769203, 0.014933278784155846, 0.023366808891296387, 0.015231836587190628, 0.025233011692762375, 0.01786077581346035, 0.019103143364191055, 0.013185148127377033, 0.017413634806871414]

    sample = samples[dataset]

print()
print("SAWO")
print(sample)
print("avg", stats.mean(sample))
print("std", np.std(sample))
print("min", min(sample))
print("max", max(sample))
print()



SAWO
[0.9790209531784058, 0.9930070042610168, 0.9790209531784058, 0.9790209531784058, 0.9860140085220337, 0.9790209531784058, 0.9790209531784058, 0.9790209531784058, 0.9930070042610168, 0.9440559148788452, 0.9510489702224731, 0.9650349617004395, 0.9860140085220337, 0.9720279574394226, 0.9860140085220337, 0.9580419659614563, 0.9790209531784058, 0.9230769276618958, 0.9720279574394226, 0.9790209531784058, 0.9720279574394226, 0.9650349617004395, 0.9650349617004395, 0.9790209531784058, 0.9790209531784058, 0.9650349617004395, 0.9720279574394226, 0.9650349617004395, 0.9790209531784058, 0.9580419659614563]
avg 0.9720279633998871
std 0.014444692009420994
min 0.9230769276618958
max 0.9930070042610168



In [19]:
#CREATE BASELINE SAMPLE

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

learning_rates = [None,0.23,0.18,0.22,0.18,0.50,0.48,0.48,0.39,0.26,0.22,0.34,0.14]
learning_rate = learning_rates[dataset]

base_sample = []
for i in range(sample_size):  
    print(i)

    val = np.NAN

    count = 0
    while count < 2 and np.isnan(val):
        count += 1

        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=(seed+i))

        init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.2)

        model = keras.Sequential([
            keras.Input(shape=(X_train.shape[1])),
            layers.Dense(15, kernel_initializer=init),
            layers.Dense(15, kernel_initializer=init),
            layers.Dense(15, kernel_initializer=init),
            layers.Dense(y_train.shape[1], kernel_initializer=init),
        ])


        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

        model.compile(loss='mse', metrics = ['accuracy'], optimizer=opt)

        b_s = math.ceil(X_train.shape[0]/num_batches)
        res = model.fit(x=X_train, y=y_train, epochs=num_epochs, batch_size=b_s, validation_data=(X_test, y_test))

        if class_T_reg_F:
            print(res.history['val_accuracy'])
            val = max(res.history['val_accuracy'])
            print(val)
        else:
            print(res.history['val_loss'])
            val = min(res.history['val_loss'])
            print(val)


    if np.isnan(val):
        val = np.inf
    print("->", val)

    
    base_sample.append(val)
    print()


print("BASE")
print(base_sample)
print("avg", stats.mean(base_sample))
print("std", np.std(base_sample))
print("min", min(base_sample))
print("max", max(base_sample))
print()

0
Epoch 1/20
5/5 [==============================] - 1s 37ms/step - loss: 0.2618 - accuracy: 0.6948 - val_loss: 0.1742 - val_accuracy: 0.8112
Epoch 2/20
5/5 [==============================] - 0s 7ms/step - loss: 0.1653 - accuracy: 0.8263 - val_loss: 0.1262 - val_accuracy: 0.8671
Epoch 3/20
5/5 [==============================] - 0s 7ms/step - loss: 0.1276 - accuracy: 0.8662 - val_loss: 0.0977 - val_accuracy: 0.8951
Epoch 4/20
5/5 [==============================] - 0s 8ms/step - loss: 0.1102 - accuracy: 0.8826 - val_loss: 0.0998 - val_accuracy: 0.9510
Epoch 5/20
5/5 [==============================] - 0s 7ms/step - loss: 0.0998 - accuracy: 0.9108 - val_loss: 0.0812 - val_accuracy: 0.8951
Epoch 6/20
5/5 [==============================] - 0s 8ms/step - loss: 0.0939 - accuracy: 0.9014 - val_loss: 0.0906 - val_accuracy: 0.9510
Epoch 7/20
5/5 [==============================] - 0s 8ms/step - loss: 0.0930 - accuracy: 0.9178 - val_loss: 0.0946 - val_accuracy: 0.8531
Epoch 8/20
5/5 [===============

In [20]:
from scipy.stats import ttest_rel

print("PAIRED T TEST")
t, p = ttest_rel(base_sample.copy(), sample.copy())
print('t=%.10f, p=%.10f' % (t, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')
 

alpha = 0.5
print("Confidence:", ((1.0 - alpha)*100), "%")


if class_T_reg_F:

    if p / 2 < alpha and t > 0:
        print("BASE > BEST i.e. BASE wins!")
        print("Will win at %.6f" % ((1.0 - (p/2))*100), "% confidence with p/2 =", "%.6f" % (p/2))
    else:
        print("BASE <= BEST i.e. no clear winner")

    if p / 2 < alpha and t < 0:
        print("BASE < BEST i.e. BEST wins!")
        print("Will win at %.6f" % ((1.0 - (p/2))*100), "% confidence with p/2 =", "%.6f" % (p/2))
    else:
        print("BASE >= BEST i.e. no clear winner")

else:

    if p / 2 < alpha and t > 0:
        print("BASE > BEST i.e. BEST wins!")
        print("Will win at %.6f" % ((1.0 - (p/2))*100), "% confidence with p/2 =", "%.6f" % (p/2))
    else:
        print("BASE <= BEST i.e. no clear winner")

    if p / 2 < alpha and t < 0:
        print("BASE < BEST i.e. BASE wins!")
        print("Will win at %.6f" % ((1.0 - (p/2))*100), "% confidence with p/2 =", "%.6f" % (p/2))
    else:
        print("BASE >= BEST i.e. no clear winner")



PAIRED T TEST
t=-3.3397348102, p=0.0023162446
Probably different distributions
Confidence: 50.0 %
BASE <= BEST i.e. no clear winner
BASE < BEST i.e. BEST wins!
Will win at 99.884188 % confidence with p/2 = 0.001158
